In [ ]:
PROTGPS_PARENT_DIR = "/home/shenrunx/igvf/varchamp/protgps_varchamp" # point to the protgps local repo

# !pip install fsspec==2023.5.0
# pip install transformers

In [ ]:
## Avoid some weird errors
# File ~/igvf/varchamp/protgps_varchamp/protgps/lightning/base.py:2
#       1 import torch
# ----> 2 import pytorch_lightning as pl
#       3 import torch.nn.functional as F
#       4 import numpy as np

# File ~/software/anaconda3/envs/protgps/lib/python3.8/site-packages/pytorch_lightning/__init__.py:30
#      27     _logger.addHandler(logging.StreamHandler())
#      28     _logger.propagate = False
# ---> 30 from pytorch_lightning.callbacks import Callback  # noqa: E402
#      31 from pytorch_lightning.core import LightningDataModule, LightningModule  # noqa: E402
#      32 from pytorch_lightning.trainer import Trainer  # noqa: E402

# File ~/software/anaconda3/envs/protgps/lib/python3.8/site-packages/pytorch_lightning/callbacks/__init__.py:22
#      20 from pytorch_lightning.callbacks.lambda_function import LambdaCallback
#      21 from pytorch_lightning.callbacks.lr_monitor import LearningRateMonitor
# ---> 22 from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
#      23 from pytorch_lightning.callbacks.model_summary import ModelSummary
#      24 from pytorch_lightning.callbacks.prediction_writer import BasePredictionWriter

# File ~/software/anaconda3/envs/protgps/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:37
#      35 import pytorch_lightning as pl
#      36 from pytorch_lightning.callbacks.base import Callback
# ---> 37 from pytorch_lightning.utilities.cloud_io import get_filesystem
#      38 from pytorch_lightning.utilities.exceptions import MisconfigurationException
#      39 from pytorch_lightning.utilities.logger import _name, _version

# File ~/software/anaconda3/envs/protgps/lib/python3.8/site-packages/pytorch_lightning/utilities/cloud_io.py:20
#      17 from pathlib import Path
#      18 from typing import Any, Callable, Dict, IO, Optional, Union
# ---> 20 import fsspec
#      21 import torch
#      22 from fsspec.core import url_to_fs

# File ~/software/anaconda3/envs/protgps/lib/python3.8/site-packages/fsspec/__init__.py:3
#       1 from importlib.metadata import entry_points
# ----> 3 from . import caching
#       4 from ._version import __version__  # noqa: F401
#       5 from .callbacks import Callback

# File ~/software/anaconda3/envs/protgps/lib/python3.8/site-packages/fsspec/caching.py:40
#      37 logger = logging.getLogger("fsspec")
#      39 Fetcher = Callable[[int, int], bytes]  # Maps (start, end) to bytes
# ---> 40 MultiFetcher = Callable[list[[int, int]], bytes]  # Maps [(start, end)] to bytes
#      43 class BaseCache:
#      44     """Pass-though cache: doesn't keep anything, calls every time
#      45 
#      46     Acts as base class for other cachers
#    (...)
#      56         How big this file is
#      57     """

# TypeError: 'type' object is not subscriptable

In [25]:
import sys
import os
sys.path.append(PROTGPS_PARENT_DIR) # append the path of protgps
from argparse import Namespace
import pickle
from tqdm import tqdm
import pandas as pd
import torch
from protgps.utils.loading import get_object

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [26]:
COMPARTMENT_CLASSES = [
    "nuclear_speckle",
    "p-body",
    "pml-bdoy",
    "post_synaptic_density",
    "stress_granule",
    "chromosome",
    "nucleolus",
    "nuclear_pore_complex",
    "cajal_body",
    "rna_granule",
    "cell_junction",
    "transcriptional"
]


def load_model(snargs):
    """
    Loads classifier model from args file
    """
    modelpath = snargs.model_path
    model = get_object(snargs.lightning_name, "lightning")(snargs)
    model = model.load_from_checkpoint(
        checkpoint_path = modelpath,
        strict=not snargs.relax_checkpoint_matching,
        **{"args": snargs},
    )
    return model


@torch.no_grad()
def predict_condensates(model, sequences, batch_size=1, round=True):
    scores = []
    for i in tqdm(range(0, len(sequences), batch_size), ncols=100):
        batch = sequences[ i : (i + batch_size)]
        out = model.model({"x": batch})    
        s = torch.sigmoid(out['logit']).to("cpu")
        scores.append(s)
    scores = torch.vstack(scores)
    if round:
        scores = torch.round(scores, decimals=3)
    return scores

In [29]:
args = Namespace(**pickle.load(open(os.path.join(PROTGPS_PARENT_DIR, 'checkpoints/protgps/32bf44b16a4e770a674896b81dfb3729.args'),'rb'))) # assumes args file has been extracted in checkpoints/protgps
args.model_path = os.path.join(PROTGPS_PARENT_DIR, 'checkpoints/protgps/32bf44b16a4e770a674896b81dfb3729epoch=26.ckpt') # assumes checkpoint has been extracted in checkpoints/protgps
args.pretrained_hub_dir = os.path.join(PROTGPS_PARENT_DIR, "checkpoints/esm2/facebookresearch_esm_main") # should point to folder with ESM2 facebookresearch_esm_main directory

model = load_model(args)
model.eval()
model = model.to(device)

Exception: INVALID LIGHTNING NAME: base. AVAILABLE dict_keys([])

In [28]:
args = Namespace(**pickle.load(open(os.path.join(PROTGPS_PARENT_DIR, 'checkpoints/protgps/7c4853cd22080b250ef89af2a1b25102.args'),'rb'))) # assumes args file has been extracted in checkpoints/protgps
args.model_path = os.path.join(PROTGPS_PARENT_DIR, 'checkpoints/protgps/7c4853cd22080b250ef89af2a1b25102epoch=3.ckpt') # assumes checkpoint has been extracted in checkpoints/protgps
args.pretrained_hub_dir = os.path.join(PROTGPS_PARENT_DIR, "checkpoints/esm2/facebookresearch_esm_main") # should point to folder with ESM2 facebookresearch_esm_main directory

# print(args.lightning_name)
model = load_model(args)
model.eval()
model = model.to(device)

Exception: INVALID LIGHTNING NAME: base. AVAILABLE dict_keys([])

In [3]:
sequences = [
    # UniProt O15116
    "MNYMPGTASLIEDIDKKHLVLLRDGRTLIGFLRSIDQFANLVLHQTVERIHVGKKYGDIPRGIFVVRGENVVLLGEIDLEKESDTPLQQVSIEEILEEQRVEQQTKLEAEKLKVQALKDRGLSIPRADTLDEY", 
    # Uniprot P38432
    "MAASETVRLRLQFDYPPPATPHCTAFWLLVDLNRCRVVTDLISLIRQRFGFSSGAFLGLYLEGGLLPPAESARLVRDNDCLRVKLEERGVAENSVVISNGDINLSLRKAKKRAFQLEEGEETEPDCKYSKKHWKSRENNNNNEKVLDLEPKAVTDQTVSKKNKRKNKATCGTVGDDNEEAKRKSPKKKEKCEYKKKAKNPKSPKVQAVKDWANQRCSSPKGSARNSLVKAKRKGSVSVCSKESPSSSSESESCDESISDGPSKVTLEARNSSEKLPTELSKEEPSTKNTTADKLAIKLGFSLTPSKGKTSGTTSSSSDSSAESDDQCLMSSSTPECAAGFLKTVGLFAGRGRPGPGLSSQTAGAAGWRRSGSNGGGQAPGASPSVSLPASLGRGWGREENLFSWKGAKGRGMRGRGRGRGHPVSCVVNRSTDNQRQQQLNDVVKNSSTIIQNPVETPKKDYSLLPLLAAAPQVGEKIAFKLLELTSSYSPDVSDYKEGRILSHNPETQQVDIEILSSLPALREPGKFDLVYHNENGAEVVEYAVTQESKITVFWKELIDPRLIIESPSNTSSTEPA" 
]

In [ ]:
scores = predict_condensates(model, sequences, batch_size=1)

In [ ]:
data = {"sequences": sequences}
for j,condensate in enumerate(COMPARTMENT_CLASSES):
    data[f"{condensate.upper()}_Score"] = scores[:, j].tolist()

In [ ]:
pd.DataFrame(data)